## Distributed Arrays

In [1]:
using Distributed

In [2]:
nprocs() == 1 && addprocs(;exeflags="--project")

8-element Vector{Int64}:
 2
 3
 4
 5
 6
 7
 8
 9

In [3]:
@everywhere using DistributedArrays

In [4]:
workers()

8-element Vector{Int64}:
 2
 3
 4
 5
 6
 7
 8
 9

In [5]:
dat=rand(10000,10000)

10000×10000 Matrix{Float64}:
 0.915784  0.750595   0.299321    …  0.346827   0.597206  0.667074
 0.798144  0.895753   0.76358        0.810244   0.313943  0.188007
 0.796687  0.793964   0.357841       0.87284    0.080122  0.135463
 0.853148  0.187906   0.590515       0.17427    0.060214  0.172934
 0.620562  0.37778    0.282434       0.0502126  0.917666  0.396084
 0.509029  0.805374   0.593719    …  0.928261   0.973436  0.986636
 0.172196  0.791609   0.756731       0.698614   0.441773  0.573529
 0.453831  0.638059   0.0679059      0.467407   0.902078  0.426939
 0.604163  0.128133   0.861213       0.128442   0.746511  0.259181
 0.086356  0.211106   0.777845       0.272174   0.676648  0.410222
 0.542005  0.904794   0.768647    …  0.544643   0.139152  0.761372
 0.349969  0.0901272  0.741803       0.742769   0.815981  0.455522
 0.461149  0.75647    0.977644       0.728967   0.837849  0.984041
 ⋮                                ⋱                       
 0.158797  0.474288   0.0652504      0.37

In [6]:
map(x->myid(),dat)

10000×10000 Matrix{Int64}:
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  

In [7]:
ddat=distribute(dat)

10000×10000 DArray{Float64, 2, Matrix{Float64}}:
 0.915784  0.750595   0.299321    …  0.346827   0.597206  0.667074
 0.798144  0.895753   0.76358        0.810244   0.313943  0.188007
 0.796687  0.793964   0.357841       0.87284    0.080122  0.135463
 0.853148  0.187906   0.590515       0.17427    0.060214  0.172934
 0.620562  0.37778    0.282434       0.0502126  0.917666  0.396084
 0.509029  0.805374   0.593719    …  0.928261   0.973436  0.986636
 0.172196  0.791609   0.756731       0.698614   0.441773  0.573529
 0.453831  0.638059   0.0679059      0.467407   0.902078  0.426939
 0.604163  0.128133   0.861213       0.128442   0.746511  0.259181
 0.086356  0.211106   0.777845       0.272174   0.676648  0.410222
 0.542005  0.904794   0.768647    …  0.544643   0.139152  0.761372
 0.349969  0.0901272  0.741803       0.742769   0.815981  0.455522
 0.461149  0.75647    0.977644       0.728967   0.837849  0.984041
 ⋮                                ⋱                       
 0.158797  0.474288  

In [8]:
map(x->myid(),ddat)

10000×10000 DArray{Int64, 2, Matrix{Int64}}:
 2  2  2  2  2  2  2  2  2  2  2  2  2  …  8  8  8  8  8  8  8  8  8  8  8  8
 2  2  2  2  2  2  2  2  2  2  2  2  2     8  8  8  8  8  8  8  8  8  8  8  8
 2  2  2  2  2  2  2  2  2  2  2  2  2     8  8  8  8  8  8  8  8  8  8  8  8
 2  2  2  2  2  2  2  2  2  2  2  2  2     8  8  8  8  8  8  8  8  8  8  8  8
 2  2  2  2  2  2  2  2  2  2  2  2  2     8  8  8  8  8  8  8  8  8  8  8  8
 2  2  2  2  2  2  2  2  2  2  2  2  2  …  8  8  8  8  8  8  8  8  8  8  8  8
 2  2  2  2  2  2  2  2  2  2  2  2  2     8  8  8  8  8  8  8  8  8  8  8  8
 2  2  2  2  2  2  2  2  2  2  2  2  2     8  8  8  8  8  8  8  8  8  8  8  8
 2  2  2  2  2  2  2  2  2  2  2  2  2     8  8  8  8  8  8  8  8  8  8  8  8
 2  2  2  2  2  2  2  2  2  2  2  2  2     8  8  8  8  8  8  8  8  8  8  8  8
 2  2  2  2  2  2  2  2  2  2  2  2  2  …  8  8  8  8  8  8  8  8  8  8  8  8
 2  2  2  2  2  2  2  2  2  2  2  2  2     8  8  8  8  8  8  8  8  8  8  8  8
 2  2  2  2  2  2  

In [9]:
[ @spawnat p sum(x->x*x,localpart(ddat)) for p in workers() ]

8-element Vector{Future}:
 Future(2, 1, 5108, nothing)
 Future(3, 1, 5109, nothing)
 Future(4, 1, 5110, nothing)
 Future(5, 1, 5111, nothing)
 Future(6, 1, 5112, nothing)
 Future(7, 1, 5113, nothing)
 Future(8, 1, 5114, nothing)
 Future(9, 1, 5115, nothing)

In [10]:
map(fetch,[ @spawnat p sum(x->x*x,localpart(ddat)) for p in workers() ])

8-element Vector{Float64}:
 4.1679458340829625e6
 4.1648904531229585e6
 4.1680942687513027e6
 4.1679629642029707e6
 4.1669420137302354e6
 4.167556850732112e6
 4.1676458410920156e6
 4.167688895582674e6

In [11]:
@time reduce(+,map(fetch,[ @spawnat p sum(x->x*x,localpart(ddat)) for p in workers() ]))

  0.200300 seconds (316.66 k allocations: 18.853 MiB, 57.24% compilation time)


3.3338727121297233e7

In [12]:
@time sum(ddat .* ddat)

  1.616504 seconds (1.54 M allocations: 92.914 MiB, 31.71% compilation time)


3.3338727121297233e7

In [13]:
@code_native sum(ddat .* ddat)

	.section	__TEXT,__text,regular,pure_instructions
; ┌ @ reducedim.jl:873 within `sum'
	subq	$40, %rsp
	movq	%rsi, 32(%rsp)
	movq	(%rsi), %rax
	movabsq	$jl_system_image_data, %rcx
; │┌ @ reducedim.jl:873 within `#sum#679'
; ││┌ @ reducedim.jl:877 within `_sum'
; │││┌ @ reducedim.jl:877 within `#_sum#681'
; ││││┌ @ reducedim.jl:878 within `_sum'
; │││││┌ @ reducedim.jl:878 within `#_sum#682'
; ││││││┌ @ reducedim.jl:310 within `mapreduce'
; │││││││┌ @ reducedim.jl:310 within `#mapreduce#672'
; ││││││││┌ @ reducedim.jl:318 within `_mapreduce_dim'
	movq	%rcx, (%rsp)
	movabsq	$jl_system_image_data, %rcx
	movq	%rcx, 8(%rsp)
	movabsq	$jl_system_image_data, %rcx
	movq	%rcx, 16(%rsp)
	movq	%rax, 24(%rsp)
	movabsq	$jl_invoke, %rax
	movabsq	$jl_system_image_data, %rdi
	movq	%rsp, %rsi
	movabsq	$6171592112, %rcx               ## imm = 0x16FDB05B0
	movl	$4, %edx
	callq	*%rax
; │└└└└└└└└
	addq	$40, %rsp
	retq
; └


In [14]:
methodswith(DArray)

[1] rand!(A::DArray, ::Type{T}) where T in DistributedArrays at /Users/ppalmes/.julia/packages/DistributedArrays/3FSDt/src/darray.jl:849
[2] nnz(A::DArray) in DistributedArrays at /Users/ppalmes/.julia/packages/DistributedArrays/3FSDt/src/mapreduce.jl:123
[3] procs(d::DArray) in DistributedArrays at /Users/ppalmes/.julia/packages/DistributedArrays/3FSDt/src/core.jl:58
[4] rem(A::Array{T, N} where N, B::DArray{T, N, A} where {N, A}) where T in DistributedArrays at /Users/ppalmes/.julia/packages/DistributedArrays/3FSDt/src/mapreduce.jl:195
[5] rem(A::DArray{T, N, A} where {N, A}, B::Array{T, N} where N) where T in DistributedArrays at /Users/ppalmes/.julia/packages/DistributedArrays/3FSDt/src/mapreduce.jl:194
[6] rem(A::DArray{T, N, A} where {N, A}, B::DArray{T, N, A} where {N, A}) where T in DistributedArrays at /Users/ppalmes/.julia/packages/DistributedArrays/3FSDt/src/mapreduce.jl:190
[7] &(A::Array{T, N} where N, B::DArray{T, N, A} where {N, A}) where T in DistributedArrays at /Users/ppalmes/.julia/packages/DistributedArrays/3FSDt/src/mapreduce.jl:195
[8] &(A::DArray{T, N, A} where {N, A}, B::Array{T, N} where N) where T in DistributedArrays at /Users/ppalmes/.julia/packages/DistributedArrays/3FSDt/src/mapreduce.jl:194
[9] &(A::DArray{T, N, A} where {N, A}, B::DArray{T, N, A} where {N, A}) where T in DistributedArrays at /Users/ppalmes/.julia/packages/DistributedArrays/3FSDt/src/mapreduce.jl:190
[10] +(A::DArray{T, N, A} where {N, A}, B::Array{T, N} where N) where T in DistributedArrays at /Users/ppalmes/.julia/packages/DistributedArrays/3FSDt/src/mapreduce.jl:194
[11] +(A::DArray{T, N, A} where {N, A}, B::DArray{T, N, A} where {N, A}) where T in DistributedArrays at /Users/ppalmes/.julia/packages/DistributedArrays/3FSDt/src/mapreduce.jl:190
[12] +(A::Array{T, N} where N, B::DArray{T, N, A} where {N, A}) where T in DistributedArrays at /Users/ppalmes/.julia/packages/DistributedArrays/3FSDt/src/mapreduce.jl:195
[13] -(D::DArray) in DistributedArrays at /Users/ppalmes/.julia/packages/DistributedArrays/3FSDt/src/mapreduce.jl:142
[14] -(A::DArray{T, N, A} where {N, A}, B::Array{T, N} where N) where T in DistributedArrays at /Users/ppalmes/.julia/packages/DistributedArrays/3FSDt/src/mapreduce.jl:194
[15] -(A::DArray{T, N, A} where {N, A}, B::DArray{T, N, A} where {N, A}) where T in DistributedArrays at /Users/ppalmes/.julia/packages/DistributedArrays/3FSDt/src/mapreduce.jl:190
[16] -(A::Array{T, N} where N, B::DArray{T, N, A} where {N, A}) where T in DistributedArrays at /Users/ppalmes/.julia/packages/DistributedArrays/3FSDt/src/mapreduce.jl:195
[17] ==(d1::DArray, d2::SubArray{T, N, D, I, L} where {T, N, D<:DArray, I, L}) in DistributedArrays at /Users/ppalmes/.julia/packages/DistributedArrays/3FSDt/src/darray.jl:444
[18] ==(d1::DArray, d2::DArray) in DistributedArrays at /Users/ppalmes/.julia/packages/DistributedArrays/3FSDt/src/darray.jl:437
[19] ==(d::DArray{var"#s37", var"#s29", A} where {var"#s37", var"#s29"}, a::AbstractArray) where A in DistributedArrays at /Users/ppalmes/.julia/packages/DistributedArrays/3FSDt/src/darray.jl:417
[20] ==(d1::SubArray{T, N, D, I, L} where {T, N, D<:DArray, I, L}, d2::DArray) in DistributedArrays at /Users/ppalmes/.julia/packages/DistributedArrays/3FSDt/src/darray.jl:438
[21] ==(a::AbstractArray, d::DArray) in DistributedArrays at /Users/ppalmes/.julia/packages/DistributedArrays/3FSDt/src/darray.jl:435
[22] close(d::DArray) in DistributedArrays at /Users/ppalmes/.julia/packages/DistributedArrays/3FSDt/src/core.jl:34
[23] copyto!(dest::DArray, bc::Base.Broadcast.Broadcasted) in DistributedArrays at /Users/ppalmes/.julia/packages/DistributedArrays/3FSDt/src/broadcast.jl:66
[24] count(f, A::DArray) in DistributedArrays at /Users/ppalmes/.julia/packages/DistributedArrays/3FSDt/src/mapreduce.jl:114
[25] deepcopy(src::DArray) in DistributedArrays at /Users/ppalmes/.julia/packages/DistributedArrays/3FSDt/src/darray.jl:688
[26] div(A::Array{T, N} where N, B::DArray{T, N, A} where {N, A}) where T in 

In [15]:
@which sum(ddat .* ddat)

sum(a::AbstractArray; dims, kw...) in Base at reducedim.jl:873